# Quality Assessment

Compute EGG quality metrics and run the automated quality check using
`cycle_stats`, `proportion_normogastric`, `instability_coefficient`,
and `assess_quality`.

In [1]:
import gastropy as gp

# Load and process
egg = gp.load_egg()
best_idx, _, _, _ = gp.select_best_channel(egg["signal"], egg["sfreq"])
_, info = gp.egg_process(egg["signal"][best_idx], egg["sfreq"])
durations = info["cycle_durations_s"]

## Cycle Statistics

In [2]:
c_stats = gp.cycle_stats(durations)
for key, val in c_stats.items():
    if isinstance(val, float):
        print(f"  {key:20s}: {val:.3f}")
    else:
        print(f"  {key:20s}: {val}")

  n_cycles            : 37
  mean_cycle_dur_s    : 20.289
  sd_cycle_dur_s      : 3.569
  prop_within_3sd     : 0.946
  lower_3sd_s         : 9.582
  upper_3sd_s         : 30.997


## Individual Metrics

In [3]:
prop_normo = gp.proportion_normogastric(durations)
ic = gp.instability_coefficient(durations)

print(f"Proportion normogastric: {prop_normo:.0%}")
print(f"Instability coefficient: {ic:.4f}")

Proportion normogastric: 95%
Instability coefficient: 0.1759


## Automated Quality Check

`assess_quality` applies standard thresholds:
- At least 10 cycles
- Cycle SD < 6 s
- At least 70% normogastric
- Overall: sufficient cycles AND (stable OR normogastric)

In [4]:
qc = gp.assess_quality(
    n_cycles=c_stats["n_cycles"],
    cycle_sd=c_stats["sd_cycle_dur_s"],
    prop_normo=prop_normo,
)

for key, val in qc.items():
    print(f"  {key:24s}: {'PASS' if val else 'FAIL'}")

  sufficient_cycles       : PASS
  stable_rhythm           : PASS
  normogastric_dominant   : PASS
  overall                 : PASS


## Band Power Metrics

In [5]:
bp = info["band_power"]
print(f"Peak frequency:    {bp['peak_freq_hz']:.4f} Hz")
print(f"Max power:         {bp['max_power']:.6f}")
print(f"Mean power:        {bp['mean_power']:.6f}")
print(f"Power proportion:  {bp['prop_power']:.1%}")
print(f"Mean power ratio:  {bp['mean_power_ratio']:.2f}")

Peak frequency:    0.0530 Hz
Max power:         0.000000
Mean power:        0.000000
Power proportion:  80.3%
Mean power ratio:  2.21


**See also:** [Cycle Histogram](plot_cycle_histogram.ipynb),
[Artifact Detection](plot_artifacts.ipynb)